In [ ]:
#@title Prevent disconnections
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
# @title Load utils

globals()["SOURCE"] = "oobabooga"
globals()["NAME"] = "text-generation-webui"

BASE = f"https://github.com/{globals()['SOURCE']}/{globals()['NAME']}"
BASE_PATH = f"/content/{globals()['NAME']}"
# NGROK = f"/content/{globals()['NAME']}/extensions/ngrok/requirements.txt"


def get_renamed_file(MODEL_TYPE, MODEL_FILES, links):
    if MODEL_TYPE == "gguf":
        truth_links = MODEL_FILES if len(MODEL_FILES) > 0 else links
        file = list(filter(lambda filename: filename.endswith(".gguf"), truth_links))[0]
        return file.split(file[file.rfind("/")])[-1]

    if MODEL_TYPE == "gptq":
        return "gptq_model-4bit-128g.safetensors"

    if MODEL_TYPE == "exl2":
        return "output.safetensors"
    

def get_target_repo(MODEL_ID):
    return f"https://huggingface.co/{MODEL_ID}/tree/main?not-for-all-audiences=true"


def get_target_links(MODEL_TYPE, MODEL_FILES, links):

    LINKS = []

    if MODEL_TYPE == "gguf":
        truth_links = MODEL_FILES if len(MODEL_FILES) > 0 else links
        LINKS = [link.replace("/blob/", "/resolve/") for link in truth_links if link.endswith(".gguf") and "/blob/" in link]
        return LINKS

    FILE_TARGETS = (".json", ".model", ".safetensors", ".yml", ".py")

    if len(MODEL_FILES) > 0:
        LINKS = MODEL_FILES
        for item in LINKS:
            item = item.replace("/blob/", "/resolve/")
    else:
        for link in links:
            if (
                any(link.endswith(target) for target in FILE_TARGETS)
                and "/resolve/" in link
            ):
                LINKS.append(link)

    return LINKS


def create_download_file(MODEL_TYPE, LINKS, shopping_list, renamed_file):
    with open(shopping_list, "a") as file:
        if MODEL_TYPE == "gguf":
            for url in LINKS:
                file.write(f"{url}\n out={renamed_file}\n")
        else:
            for url in LINKS:
                if url.endswith(".safetensors"):
                    file.write(f"{url}\n out={renamed_file}\n")
                elif url.endswith(".model"):
                    file.write(f"{url}\n out=tokenizer.model\n")
                else:
                    file.write(f"{url}\n")

In [ ]:
#@title Download & install backend
!apt-get -y install -qq aria2
#!pip install flask-cloudflared
!pip install requests-html

!git clone $BASE
%cd $BASE_PATH
!pip install -r requirements.txt
#!pip install -r $NGROK

In [ ]:
#@title Add model

%cd /content/

from requests_html import HTMLSession

OPTIONS = {
    1: {
        "id": "TheBloke/Echidna-13B-v0.3-GPTQ", 
        "file_list": []
    },
    2: {
        "id": "mradermacher/Fimbulvetr-11B-v2-GGUF",
        "file_list": [
            "https://huggingface.co/mradermacher/Fimbulvetr-11B-v2-GGUF/blob/main/Fimbulvetr-11B-v2.Q5_K_S.gguf"
        ]
    }
}

#@markdown ### Model presets
Preset = "2" #@param [1,2]

OPTION = OPTIONS[int(Preset)]
MODEL_ID = OPTION["id"]
MODEL_FILES = OPTION["file_list"]
MODEL_NAME = MODEL_ID.split(MODEL_ID[MODEL_ID.rfind("/")])[-1]

#@markdown ### Select model type
MODEL_TYPE = "gguf" #@param ["gguf", "gtpq", "exl2"]

 #Target repo
REPO = get_target_repo(MODEL_ID)

#Get file urls
session = HTMLSession()
result = session.get(REPO)
links = result.html.absolute_links

#@markdown ### model file name based on backend and model type (empty by default)
renamed_file = "" #@param {type:"string"}
renamed_file = renamed_file or get_renamed_file(MODEL_TYPE, MODEL_FILES, links)

#Set formatted LINKS list
LINKS = get_target_links(MODEL_TYPE, MODEL_FILES, links)

#Input file for aria2
shopping_list = "down_list.txt"

#Create file for aria2 using LINKS list
create_download_file(MODEL_TYPE, LINKS, shopping_list, renamed_file)

#Set models destination for backend
models_folder = f"{BASE_PATH}/models"

#Download all model config files
print(f"Downloading {MODEL_NAME}...\n")

if MODEL_TYPE == "gguf":
    !cd $models_folder
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M --summary-interval=5 --input-file=$shopping_list -d $models_folder
    !rm $shopping_list

if MODEL_TYPE == "gptq":
    !cd $models_folder && mkdir $MODEL_NAME
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M --summary-interval=5 --input-file=$shopping_list -d $models_folder/$MODEL_NAME
    !rm $shopping_list

In [ ]:
#@title LAUNCH! (Run again in case of cloudflare error)

%cd $BASE_PATH

provider = "cloudflare" #@param ["ngrok", "cloudflare"]
context = "4096" #@param {type:"string"}
gpu_layers = "60" #@param {type:"string"}

CONTEXT = int(context)
GPU_LAYERS = int(gpu_layers)

if provider == "ngrok":
    PROVIDER = "--extension ngrok"
else:
    PROVIDER = ""

if MODEL_TYPE == "gguf":
    LOADER = "LLAMACPP"
    MODEL_NAME = renamed_file
    !python server.py --share --nowebui --api --public-api $PROVIDER --model $MODEL_NAME --loader $LOADER --max_seq_len $CONTEXT --n_ctx $CONTEXT --n-gpu-layers $GPU_LAYERS

if MODEL_TYPE == "gptq":
    LOADER = "EXLLAMAV2_HF"
    !python server.py --share --nowebui --api --public-api $PROVIDER --model $MODEL_NAME --loader $LOADER --max_seq_len $CONTEXT --n_ctx $CONTEXT